# Web Scraping- comparando precios de libros 

## Usaremos dos paginas web 

### Primero cargamos las librerias necesarias

In [2]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from tabulate import tabulate

### Preparar y configurar Google Chrome

In [3]:
# Funcion que ejecuta Chrome en segundo plano
# Evita el cierre al identificarse como usuario(no bot)
# Descarga y actualiza automáticamente el "driver" (python y chrome)
def configurar_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

### Generamos una funcion para extraer los datos importantes de cada pagina web

In [ ]:
# Usamos las paginas web: 
# https://www.sbs.com.pe 
# https://www.crisol.com.pe

In [5]:
def extraer_datos(driver, wait, busqueda, url_base, tienda_nombre):
    print(f"\n Buscando todos los resultados en {tienda_nombre}...")
    driver.get(url_base)
    
    try:
        search_box = wait.until(EC.element_to_be_clickable((By.ID, "search")))
        search_box.clear()
        search_box.send_keys(busqueda + Keys.ENTER)
        
        # Esperar a que los productos carguen
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-item-link")))
        
        # Hacer scroll hacia abajo para cargar todos los libros dinámicos
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 

        elementos = driver.find_elements(By.CLASS_NAME, "product-item-link")
        # Capturamos todos los enlaces encontrados sin límite artificial
        urls = [el.get_attribute('href') for el in elementos]
        print(f" Se detectaron {len(urls)} libros en {tienda_nombre}.")
        
    except Exception as e:
        print(f" Error al buscar en {tienda_nombre}: {e}")
        return []

    resultados = []
    for i, url in enumerate(urls):
        try:
            driver.get(url)
            time.sleep(1.2) 
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            titulo = soup.find('h1', class_='page-title').text.strip() if soup.find('h1', class_='page-title') else "N/A"
            precio_text = soup.find('span', class_='price').text.strip() if soup.find('span', class_='price') else "S/ 0.00"
            
            detalles = {"Autor": "N/A", "Año": "N/A", "ISBN": "N/A"}
            tabla = soup.find('table', id='product-attribute-specs-table')
            if tabla:
                for fila in tabla.find_all('tr'):
                    th = fila.find('th').text.strip().lower()
                    td = fila.find('td').text.strip()
                    if 'autor' in th: detalles["Autor"] = td
                    if 'año' in th or 'publicación' in th: detalles["Año"] = td
                    if 'isbn' in th: detalles["ISBN"] = td.replace('-', '').replace(' ', '')

            estado = "Disponible" if driver.find_elements(By.ID, "product-addtocart-button") else "Sin Stock"

            resultados.append({
                'Título': titulo[:40],
                'Autor': detalles["Autor"],
                'ISBN': detalles["ISBN"],
                'Año': detalles["Año"],
                'Precio': precio_text,
                'Precio_Num': float(precio_text.replace('S/', '').replace(',', '').strip()) if 'S/' in precio_text else 0.0,
                'Estado': estado
            })
            print(f"[{i+1}/{len(urls)}] Procesado: {titulo[:25]}...")
        except:
            continue
            
    return resultados

### Generamos los datos de busqueda en cada pagina,los mostramos y almacenamos

In [6]:
# --- FLUJO PRINCIPAL ---
libro_query = input("Escribe el nombre del libro a buscar: ")
driver = configurar_driver()
wait = WebDriverWait(driver, 15)

try:
    # 1. Scraping SBS
    data_sbs = extraer_datos(driver, wait, libro_query, "https://www.sbs.com.pe/", "SBS")
    df_sbs = pd.DataFrame(data_sbs)
    
    # 2. Scraping CRISOL
    data_crisol = extraer_datos(driver, wait, libro_query, "https://www.crisol.com.pe/", "CRISOL")
    df_crisol = pd.DataFrame(data_crisol)

    # TABLA SBS ---
    if not df_sbs.empty:
        print("\n" + "—"*40 + " TABLA SBS (TODOS) " + "—"*40)
        print(tabulate(df_sbs.drop(columns=['Precio_Num']), headers='keys', tablefmt='grid'))
        df_sbs.to_csv(f"sbs_{libro_query}.csv", index=False, encoding='utf-8-sig')

    # TABLA CRISOL ---
    if not df_crisol.empty:
        print("\n" + "—"*40 + " TABLA CRISOL (TODOS) " + "—"*40)
        print(tabulate(df_crisol.drop(columns=['Precio_Num']), headers='keys', tablefmt='grid'))
        df_crisol.to_csv(f"crisol_{libro_query}.csv", index=False, encoding='utf-8-sig')

    # TABLA COMPARATIVA ---
    if not df_sbs.empty and not df_crisol.empty:
        df_sbs_c = df_sbs[df_sbs['ISBN'] != "N/A"]
        df_cri_c = df_crisol[df_crisol['ISBN'] != "N/A"]
        
        comparativa = pd.merge(df_sbs_c, df_cri_c, on='ISBN', suffixes=('_SBS', '_CRI'))

        if not comparativa.empty:
            reporte = []
            for _, f in comparativa.iterrows():
                mejor = "SBS" if f['Precio_Num_SBS'] < f['Precio_Num_CRI'] else "CRISOL"
                if f['Precio_Num_SBS'] == f['Precio_Num_CRI']: mejor = "EMPATE"
                
                reporte.append({
                    'ISBN': f['ISBN'],
                    'Título': f['Título_SBS'],
                    'Precio SBS': f['Precio_SBS'],
                    'Precio Crisol': f['Precio_CRI'],
                    'Mejor Opción': mejor
                })
            
            df_comp = pd.DataFrame(reporte)
            print("\n" + "COMPARATIVA FINAL")
            print(tabulate(df_comp, headers='keys', tablefmt='fancy_grid'))
            df_comp.to_csv(f"comparativa_{libro_query}.csv", index=False, encoding='utf-8-sig')

finally:
    driver.quit()
    print("\nProceso completado con éxito.")

# Para este caso buscaremos el libro llamado Pinocho en ambas paginas web mostrando la informacion de cada resultado,
# almacenandola en un csv y luego compararemos las opciones disponibles.

Escribe el nombre del libro a buscar:  pinocho



 Buscando todos los resultados en SBS...
 Se detectaron 16 libros en SBS.
[1/16] Procesado: PINOCHO...
[2/16] Procesado: PINOCHO...
[3/16] Procesado: 10 SONIDOS PINOCHO...
[4/16] Procesado: PEQUEÑOS RELATOS: PINOCHO...
[5/16] Procesado: AVENTURAS DE PINOCHO, LAS...
[6/16] Procesado: CUENTO INFANTIL 32 PÁGINA...
[7/16] Procesado: LAS AVENTURAS DE PINOCHO ...
[8/16] Procesado: REINOS ENCANTADOS: LA CEN...
[9/16] Procesado: MUNDO MÁGICO: HISTORIAS F...
[10/16] Procesado: LAS AVENTURAS DE PINOCHO...
[11/16] Procesado: LAS AVENTURAS DE PINOCHO...
[12/16] Procesado: LAS AVENTURAS DE PINOCHO ...
[13/16] Procesado: PINOCHO Y SUS AVENTURAS P...
[14/16] Procesado: CLASICOS PARA COLOREAR: A...
[15/16] Procesado: PEQUEÑOS CLÁSICOS PARA NI...
[16/16] Procesado: CLASICOS CON LENTICULAR: ...

 Buscando todos los resultados en CRISOL...
 Se detectaron 10 libros en CRISOL.
[1/10] Procesado: Pinocho...
[2/10] Procesado: Pinocho...
[3/10] Procesado: Descubre y lee: Pinocho...
[4/10] Procesado: Pop Up. P